In [ ]:
!unzip train_test_dataset.zip -d data

In [2]:
import tensorflow as tf
from tensorflow import keras
import tensorflow.keras.utils as utils
from google.colab.patches import cv2_imshow


In [3]:
BATCH_SIZE = 32
IMG_SIZE = (256, 256)
SEED = 42
COLOR_MODE = "grayscale"

train_path = "/content/data/train_test_dataset/train"
test_path =  "/content/data/train_test_dataset/test"

In [4]:
train_data = utils.image_dataset_from_directory(train_path,subset='training' ,validation_split=0.2, seed = 42, image_size=IMG_SIZE, batch_size=BATCH_SIZE,
                                                color_mode=COLOR_MODE)
print('-------------------------')
test_data = utils.image_dataset_from_directory(test_path, seed = 42, image_size=IMG_SIZE, batch_size=BATCH_SIZE, color_mode=COLOR_MODE)


Found 200 files belonging to 5 classes.
Using 160 files for training.
-------------------------
Found 40 files belonging to 5 classes.


In [5]:
class_names = train_data.class_names
class_names

['personA', 'personB', 'personC', 'personD', 'personE']

In [6]:
batch = train_data.as_numpy_iterator().next()

In [ ]:
for i in range(len(batch[0])):
  cv2_imshow(batch[0][i])
  print(train_data.class_names[batch[1][i]])
  

In [11]:
num_classes = len(class_names)

model = keras.Sequential([
  keras.layers.Rescaling(1./255, input_shape=(*IMG_SIZE, 1)),
  keras.layers.Conv2D(8, (3,3), padding='same', activation='relu', name="CONV1"),
  keras.layers.MaxPooling2D(),
  keras.layers.Conv2D(16, (3,3), padding='same', activation='relu', name="CONV2"),
  keras.layers.MaxPooling2D(),
  keras.layers.Conv2D(32, (3,3), padding='same', activation='relu', name="CONV3"),
  keras.layers.MaxPooling2D(),
  keras.layers.Dropout(0.2),
  keras.layers.Flatten(),
  keras.layers.Dense(32, activation='relu'),
  keras.layers.Dense(num_classes)
])


In [12]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])


In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling (Rescaling)       (None, 256, 256, 1)       0         
                                                                 
 CONV1 (Conv2D)              (None, 256, 256, 8)       80        
                                                                 
 max_pooling2d (MaxPooling2D  (None, 128, 128, 8)      0         
 )                                                               
                                                                 
 CONV2 (Conv2D)              (None, 128, 128, 16)      1168      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 64, 64, 16)       0         
 2D)                                                             
                                                                 
 CONV3 (Conv2D)              (None, 64, 64, 32)        4

In [15]:
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    "checkpoints/e{epoch:02d}-acc{accuracy:.2f}.hdf5",
    monitor='accuracy',
    mode='max',
    initial_value_threshold=0.85,
    save_best_only=True)


In [16]:
epochs=7
model.fit(train_data, epochs=epochs, callbacks=[model_checkpoint_callback])

Epoch 1/7
5/5 [==============================] - 6s 944ms/step - loss: 1.6021 - accuracy: 0.2625
Epoch 2/7
5/5 [==============================] - 5s 987ms/step - loss: 1.2723 - accuracy: 0.5562
Epoch 3/7
5/5 [==============================] - 5s 936ms/step - loss: 0.8463 - accuracy: 0.8313
Epoch 4/7
5/5 [==============================] - 6s 1s/step - loss: 0.4821 - accuracy: 0.8875
Epoch 5/7
5/5 [==============================] - 5s 953ms/step - loss: 0.1735 - accuracy: 0.9875
Epoch 6/7
5/5 [==============================] - 5s 949ms/step - loss: 0.0518 - accuracy: 1.0000
Epoch 7/7
5/5 [==============================] - 5s 933ms/step - loss: 0.0150 - accuracy: 1.0000


In [21]:
model2 = keras.models.load_model('/content/checkpoints/e06-acc1.00.hdf5')

In [25]:
loss, acc = model2.evaluate(test_data, verbose=2)
print("trained model, accuracy: {:5.2f}%".format(100 * acc))

loss, acc = model.evaluate(test_data, verbose=2)
print("trained model, accuracy: {:5.2f}%".format(100 * acc))

2/2 - 0s - loss: 0.0978 - accuracy: 0.9750 - 481ms/epoch - 241ms/step
trained model, accuracy: 97.50%
2/2 - 0s - loss: 0.0454 - accuracy: 1.0000 - 497ms/epoch - 248ms/step
trained model, accuracy: 100.00%


In [27]:
model2 = keras.models.load_model('/content/good_model.hdf5')

In [28]:
loss, acc = model2.evaluate(test_data, verbose=2)
print("trained model, accuracy: {:5.2f}%".format(100 * acc))


2/2 - 1s - loss: 0.0454 - accuracy: 1.0000 - 683ms/epoch - 341ms/step
trained model, accuracy: 100.00%
